In [46]:
def multiply_by_12(x):
    y = x + x
    z = y + y
    return z + z + z
print(multiply_by_12(10))

120


In [43]:
def multiply_by_16(x):
    y = x + x
    z = y + y
    i = z + z
    return i + i
print(multiply_by_16(10))

160


In [41]:
def multiply_by_15(x):
    y = x + x
    z = y + y
    i = z + z
    m = x - i
    return i - m
print(multiply_by_15(10))

150


In [47]:
def naive_mul(x, y):
    r = 0
    for i in range(0, y):
        r = r + x
    return r

import random

for _ in range(10):
    x = random.randint(0, 100)
    y = random.randint(0, 100)
    assert naive_mul(x, y) == x * y

96
720
2813
8514
729
192
1316
395
5130
4875


In [4]:
import random

def fast_mul(x, y):
    result = 0
    while y:
        if y & 1:
            result += x
        x <<= 1
        y >>= 1
    return result

for _ in range(10):
    x = random.randint(0, 100)
    y = random.randint(0, 100)
    assert fast_mul(x, y) == x * y

In [10]:
def fast_pow(x, y):
    result = 1
    while y:
        if y & 1:
            result *= x
        x *= x
        y >>= 1
    return result

for _ in range(10):
    x = random.randint(0, 10)
    y = random.randint(0, 10)
    assert fast_pow(x, y) == x ** y

In [6]:
def mul_bits(x, y, bits):
    x &= (2 ** bits - 1)
    y &= (2 ** bits - 1)
    return x * y

def mul16(x, y):
    a, b = x >> 8, x & 0xff
    c, d = y >> 8, y & 0xff

    ac = mul_bits(a, c, 8)
    ad = mul_bits(a, d, 8)
    bc = mul_bits(b, c, 8)
    bd = mul_bits(b, d, 8)

    return (ac << 16) + ((ad + bc) << 8) + bd

import random

for _ in range(1000):
    x = random.randint(0, 2**16 - 1)
    y = random.randint(0, 2**16 - 1)
    assert mul16(x, y) == x * y

In [12]:
def mul16k(x, y):
    x0 = x & 0xff
    x1 = x >> 8
    y0 = y & 0xff
    y1 = y >> 8

    ac = x1 * y1
    bd = x0 * y0
    e = (x0 + x1) * (y0 + y1) - ac - bd

    return (ac << 16) + (e << 8) + bd


import random

for _ in range(1000):
    x = random.randint(0, 2**16 - 1)
    y = random.randint(0, 2**16 - 1)
    assert mul16k(x, y) == x * y


In [18]:
def fast_mul_gen(y):
    assignments = ["y = x"]
    for i in range(y.bit_length() - 1):
        assignments.append("y += y")
    function_body = "\n    ".join(assignments)
    function_text = f"def f(x):\n    {function_body}\n    return y"
    return function_text

# Пример использования:
y = 12
print(fast_mul_gen(y))

# Автоматическое тестирование:
for y in range(1, 11):
    exec(fast_mul_gen(y), globals())
    for x in range(1, 11):
        assert f(x) == x * y, f"Failed for x={x}, y={y}"

def f(x):
    y = x
    y += y
    y += y
    y += y
    return y


AssertionError: Failed for x=1, y=1

In [16]:
def fast_mul_gen_pow(y):
    assignments = [f"y = x"]

    for i in range(1, y):
        assignments.append(f"y *= x")

    function_body = ["\n    " + str(i) for i in assignments]
    function_text = f"def f_pow(x):{''.join(function_body)}\n    return y\n"
    return function_text


exec(fast_mul_gen_pow(y), globals())

assert fast_mul_gen_pow(x) == x ** y

AssertionError: 